In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image
from IPython.core.display import HTML 
import time
from ipywidgets import FloatProgress
from IPython.display import display
from bs4 import BeautifulSoup
import requests
import re
from os import path
from PIL import Image
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from models import *
from etl import *
import plotly.plotly as py
import plotly.graph_objs as go

Database session created


# Ketogenic

In [2]:
Limit_widget1 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget1 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget1 = widgets.Dropdown(options = ['keto'],value='keto', description ='Topic', disabled=False)
expertise_widget1 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget1 = widgets.IntSlider(min=0, max=100, step=10, value=20)


interact(query_content,Limit = Limit_widget1, Medium = Medium_widget1, search_param=topic_widget1, Formality = expertise_widget1, Max_Length = max_length_widget1);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [3]:
plot_stacked('keto')

# Paleolithic

In [4]:
Limit_widget2 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget2 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget2 = widgets.Dropdown(options = ['paleo'],value='paleo', description ='Topic', disabled=False)
expertise_widget2 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget2 = widgets.IntSlider(min=0, max=100, step=10, value=20)


interact(query_content,Limit = Limit_widget2, Medium = Medium_widget2, search_param=topic_widget2, Formality = expertise_widget2, Max_Length = max_length_widget2);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [5]:
plot_stacked('paleo')

# Vegan

In [6]:
Limit_widget3 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget3 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget3 = widgets.Dropdown(options = ['vegan'],value='vegan', description ='Topic', disabled=False)
expertise_widget3 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget3 = widgets.IntSlider(min=0, max=100, step=10, value=20)


interact(query_content,Limit = Limit_widget3, Medium = Medium_widget3, search_param=topic_widget3, Formality = expertise_widget3, Max_Length = max_length_widget3);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [7]:
plot_stacked('vegan')

# Vegetarian

In [8]:
Limit_widget4 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget4 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget4 = widgets.Dropdown(options = ['vegetarian'],value='vegetarian', description ='Topic', disabled=False)
expertise_widget4 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget4 = widgets.IntSlider(min=0, max=100, step=10, value=20)


interact(query_content,Limit = Limit_widget4, Medium = Medium_widget4, search_param=topic_widget4, Formality = expertise_widget4, Max_Length = max_length_widget4);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [9]:
plot_stacked('vegetarian')

# Search

In [10]:
# search_widget = widgets.Text(placeholder = 'Search News', description='Search:', disabled=False,continuous_update=False)
# Limit_widget5 = widgets.IntSlider(min=1, max=5, step=1, value=1)
# Medium_widget5 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')




# interact(query_content,search_param=search_widget,Limit = Limit_widget5, Medium = Medium_widget5);